# Рекомендация тарифов

## Описание проекта

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


* сalls — количество звонков,
* minutes — суммарная длительность звонков в минутах,
* messages — количество sms-сообщений,
* mb_used — израсходованный интернет-трафик в Мб,
* is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Разбейте данные на выборки

In [4]:
features = df.drop(['is_ultra'], axis = 1)
target = df['is_ultra']

In [5]:
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.4, random_state=19191)

In [6]:
features_test, features_valid, target_test, target_valid = train_test_split(
    features_test, target_test, test_size=0.5, random_state=19191)

## Исследуйте модели

### Дерево Решений

In [7]:
best_model_DTC = None
best_accuracy_DTC = 0
best_depth_DTC = 0

for depth in range(1, 11):
    model_DTC = DecisionTreeClassifier(random_state=19191, max_depth=depth)
    model_DTC.fit(features_train, target_train)

    prediction = model_DTC.predict(features_valid)
    result = accuracy_score(target_valid, prediction)
    if result > best_accuracy_DTC:
        best_model_DTC = model_DTC
        best_accuracy_DTC = result
        best_depth_DTC = depth
print("Accuracy наилучшей модели Дерево Решений:", best_accuracy_DTC, "глубина:", depth)

Accuracy наилучшей модели Дерево Решений: 0.7993779160186625 глубина: 10


### Cлучайный лес

In [8]:
best_model_RFC = None
best_accuracy_RFC = 0
best_est_RFC = 0
best_depth_RFC = 0
best_min_samples_leaf = 0
for est in range(20, 100, 10):
    for depth in range(1, 6):
        for leaf in range(1, 6):
            model_RFC = RandomForestClassifier(random_state=19191, n_estimators=est, max_depth=depth, min_samples_leaf = leaf)
            model_RFC.fit(features_train, target_train)
            prediction = model_RFC.predict(features_valid)
            result = model_RFC.score(features_valid, target_valid) 
            if result > best_accuracy_RFC:
                best_model_RFC = model_RFC
                best_accuracy_RFC = result
                best_est_RFC = est
                best_depth_RFC = depth
                min_samples_leaf = leaf
print("Accuracy наилучшей модели Cлучайный лес:", best_accuracy_RFC, "Количество деревьев:", best_est_RFC, 'Глубина:', best_depth_RFC, 'минимальное количество объектов в листе', leaf)

Accuracy наилучшей модели Cлучайный лес: 0.8087091757387247 Количество деревьев: 90 Глубина: 5 минимальное количество объектов в листе 5


### Логистическая регрессия

In [9]:
model_LR =  LogisticRegression()
model_LR.fit(features_train, target_train)
predictions_LR = model_LR.predict(features_valid)
accuracy_LR = accuracy_score(target_valid, predictions_LR)
print('Accuracy модели логистической регрессии:', accuracy_LR)

Accuracy модели логистической регрессии: 0.7060653188180405


## Проверьте модель на тестовой выборке

In [10]:
#Проверим модель: Случайный лес на тестовой выборке

final_test_predictions = best_model_RFC.predict(features_test)
final_test_result = accuracy_score(target_test, final_test_predictions)
print('Accuracy модели Случайный лес на тестовой выборке:', final_test_result)

Accuracy модели Случайный лес на тестовой выборке: 0.8040435458786936


## (бонус) Проверьте модели на адекватность

In [11]:
from sklearn.dummy import DummyClassifier
dummy_clf = DummyClassifier(strategy="most_frequent", random_state=0)
dummy_clf.fit(features_train, target_train)
dummy_clf.score(features_test, target_test)

0.6827371695178849

Сравнила модель с DummyClassifier, модель Случайный лес адекватна

# Вывод
* Обучили модель дерева решений, лучшее с глубиной = 10
* Обучили модель случайного леса. Лучшее с количеством дервьев 90, глубиной 8,минимальным количество объектов в листе 5
* Обучили модель логистической регрессии

Наиболее точный результат предсказывает модель Случайный лес с accuracy 0.8087